In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# melihat perangkat GPU
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
blok_size = 8
batch_size = 4
max_iters = 1000
# eval_interval = 2500
learning_rate = 3e-4
eval_iters = 250

cuda


In [14]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()

chars = sorted(set(text))
print(chars)
print(len(chars))
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']
81


In [15]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [16]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_bacth(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - blok_size, (batch_size,))
    #print(ix)
    x = torch.stack([data[i:i+blok_size] for i in ix])
    #print(x)
    y = torch.stack([data[i+1:i+blok_size+1] for i in ix])
    #print(y)
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_bacth('train')
print('inputs : ')
# print(x.shape)
print(x)
print('targets : ')
print(y)

inputs : 
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [33,  1, 66, 54, 64, 58,  1, 78],
        [ 1, 57, 62, 72, 73, 54, 67, 56],
        [61, 58,  1, 69, 54, 72, 72, 54]], device='cuda:0')
targets : 
tensor([[ 1,  1,  1,  1,  1,  1,  1,  1],
        [ 1, 66, 54, 64, 58,  1, 78, 68],
        [57, 62, 72, 73, 54, 67, 56, 58],
        [58,  1, 69, 54, 72, 72, 54, 60]], device='cuda:0')


In [17]:
@torch.no_grad
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_bacth(split)
            logits, loss = model(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

`@torch.no_grad()` adalah suatu konteks (context manager) yang digunakan untuk menonaktifkan perhitungan gradien selama eksekusi blok tertentu. Ini berguna ketika kita hanya ingin melakukan inferensi atau evaluasi model tanpa memperbarui bobot (weights) dan membutuhkan penyimpanan gradien.

In [18]:


x = train_data[:blok_size]
y = train_data[1:blok_size+1]

for t in range(blok_size):
    context = x[:t+1]
    target = y[t]
    print('when input is',context, 'target is', target)

when input is tensor([80]) target is tensor(1)
when input is tensor([80,  1]) target is tensor(1)
when input is tensor([80,  1,  1]) target is tensor(28)
when input is tensor([80,  1,  1, 28]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39]) target is tensor(42)
when input is tensor([80,  1,  1, 28, 39, 42]) target is tensor(39)
when input is tensor([80,  1,  1, 28, 39, 42, 39]) target is tensor(44)
when input is tensor([80,  1,  1, 28, 39, 42, 39, 44]) target is tensor(32)


In [19]:
# class BigramLanguageModel(nn.Module):
#     def __init__(self, vocab_size):
#         super().__init__()
#         self.token_embeddings_table = nn.Embedding(vocab_size, vocab_size)

#     def forward(self, index, targets):
#         logits = self.token_embeddings_table(index)
#         B,T,C = logits.shape
#         logits = logits.view(B*T,C)
#         targets = targets.view(B*T)
#         loss = F.cross_entropy(logits, targets)

#         return logits

In [20]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index)

        if targets is None:
            loss = None
        else:
            B,T,C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, index, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(index)
            # fokus only on the last time step
            logits = logits[:, -1, :] # become (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1) # (B,1)
            # append samples index to the running sequence
            index = torch.cat((index, index_next), dim=1) # (B, T+1)
        return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


b)TVQJEF﻿7"gbV,6dz[?gG0
lct )?r;aD0tzw-9oh;i.o*pjMqolJ2'J?PMuR4vOIvw(3SV_7PfBvhh6d;r!pNp-1UpCTA4cm[](P?024"Qwz:!6himUjE*a5VGVcmsZ;tI"VEvRR_,A
p-ySZ.7''G"Cup-a)
p-﻿,JIvgiFt&4VYTPa)._CEOA;4rXLjxT8kLS-40&aDkqpa4Bh6-S(&Lj[eq*H37R5a﻿9jF﻿[&4S)Yd--7o36 7vWvku;dX-(ci-g,6R)c[(8I
lKu30n!7hhA5uq8b2,c&﻿'0LUBht!ITTA.aq'IvRbc7]blyksQfIV:"V7gHGp??'Q[3saMz2]SP4B-gbe!5w9D3.TM3HrIvC30Iwot2r309K?0tkfSrgdGzEfM.H1?IPZ_lDg2?H !QN!p3JsQ)HQOv3*a-FV?nIJ.8.BT﻿hj30t(C]o1G9ppuOCvC3JneIJFxIK!k)!k9S6jzFQ&9DqdLaB;QOI,2cs5g]u.


`nn.Embedding()` adalah modul embedding layer. Embedding layer digunakan untuk mengimplementasikan representasi vektor spasial untuk token atau indeks kategori diskrit tertentu. Representasi ini disebut embedding, dan itu memungkinkan model untuk "mempelajari" representasi terbaik untuk setiap token atau kategori selama proses pelatihan.

`F.cross_entropy()` pada PyTorch adalah fungsi loss yang sering digunakan dalam tugas klasifikasi multikelas. Fungsi ini menggabungkan operasi log softmax dan negative log likelihood loss (NLLLoss) dalam satu langkah efisien.
- Log Softmax: Menerapkan operasi log softmax pada setiap prediksi, mengubah nilai-nilai prediksi menjadi skor probabilitas yang sesuai.
- Negative Log Likelihood Loss (NLLLoss): Menghitung loss sebagai negatif log likelihood dari kelas yang benar. Ini memberikan bobot yang lebih besar pada kesalahan kelas yang serius.

`F.softmax()` adalah fungsi yang digunakan untuk mengaplikasikan operasi softmax pada tensor. Softmax mengonversi nilai-nilai dalam tensor menjadi distribusi probabilitas sehingga jumlah semua probabilitasnya menjadi 1.

Softmax adalah suatu fungsi yang digunakan untuk mengonversi keluaran dari suatu model atau sistem ke dalam bentuk distribusi probabilitas. Fungsi ini digunakan khususnya dalam konteks klasifikasi, di mana model menghasilkan skor atau logit untuk setiap kelas, dan softmax kemudian mengubah skor tersebut menjadi probabilitas yang dapat diinterpretasikan.

`torch.multinomial()` adalah fungsi yang digunakan untuk mengambil sampel indeks dari distribusi multinomial. Fungsi ini sering digunakan dalam konteks pembelajaran mesin untuk memperoleh sampel berdasarkan distribusi probabilitas yang diberikan.

Fungsi ini sering digunakan dalam berbagai aplikasi, seperti saat mengambil sampel kata dari distribusi probabilitas dalam model bahasa, atau saat mengimplementasikan metode Markov Chain Monte Carlo (MCMC).

`torch.cat()` adalah fungsi yang digunakan untuk menggabungkan (concatenate) tensor di sepanjang suatu dimensi tertentu. Fungsi ini memungkinkan kita menggabungkan beberapa tensor menjadi satu tensor.

- Argumen: Argumen pertama adalah tuple dari tensor-tensor yang ingin digabungkan. Argument kedua, dim, menunjukkan dimensi di sepanjang mana penggabungan akan dilakukan.
- Dimensi: Dimensi yang digunakan untuk penggabungan harus memiliki ukuran yang sama, kecuali untuk dimensi yang digabungkan. Misalnya, jika menggabungkan tensor 2D sepanjang dimensi 0, jumlah kolom harus sama.
- Penggabungan Banyak Tensor: torch.cat() dapat digunakan untuk menggabungkan lebih dari dua tensor sekaligus dengan menyediakan tuple yang sesuai dengan jumlah tensor yang ingin digabungkan.

Fungsi ini sangat berguna dalam berbagai situasi, terutama ketika kita bekerja dengan data yang perlu digabungkan secara fleksibel, seperti dalam pengolahan gambar atau dalam pembentukan dataset untuk model pembelajaran mesin.

In [23]:
# create a pytorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step : {iter}: train loss : {losses['train']:.4f}, val loss : {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_bacth('train')

    # evaluate the loss
    logits, loss = model.forward(xb,yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

step : 0: train loss : 3.2113, val loss : 3.2183
step : 250: train loss : 3.1657, val loss : 3.1971
step : 500: train loss : 3.1291, val loss : 3.1565
step : 750: train loss : 3.1181, val loss : 3.1402
2.9627227783203125


*need to familiarize audience with optimizers (AdamW, Adam, SGD, MSE…) no need to jump into the formulas, just what the optimizer does for us and some of the differences/similarities between them*

1. Mean Squared Error (MSE): MSE is a common loss function used in regression problems, where the goal is to predict a continuous output. It measures the average squared difference between the predicted and actual values, and is often used to train neural networks for regression tasks.
2. Gradient Descent (GD): is an optimization algorithm used to minimize the loss function of a machine learning model. The loss function measures how well the model is able to predict the target variable based on the input features. The idea of GD is to iteratively adjust the model parameters in the direction of the steepest descent of the loss function
3. Momentum: Momentum is an extension of SGD that adds a "momentum" term to the parameter updates. This term helps smooth out the updates and allows the optimizer to continue moving in the right direction, even if the gradient changes direction or varies in magnitude. Momentum is particularly useful for training deep neural networks.
4. RMSprop: RMSprop is an optimization algorithm that uses a moving average of the squared gradient to adapt the learning rate of each parameter. This helps to avoid oscillations in the parameter updates and can improve convergence in some cases.
5. Adam: Adam is a popular optimization algorithm that combines the ideas of momentum and RMSprop. It uses a moving average of both the gradient and its squared value to adapt the learning rate of each parameter. Adam is often used as a default optimizer for deep learning models.
6. AdamW: AdamW is a modification of the Adam optimizer that adds weight decay to the parameter updates. This helps to regularize the model and can improve generalization performance. We will be using the AdamW optimizer as it best suits the properties of the model we will train in this video.

find more optimizers and details at torch.optim